In [2]:
import pickle as pkl
from io import BytesIO
import numpy as np
import base64
from PIL import Image, ImageFile
from pathlib import Path

from tqdm import tqdm
from multiprocessing import Pool

In [3]:
fine_id = {
    0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21,
    23: 22, 24: 23, 26: 24, 27: 25, 30: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32, 37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 45: 40, 46: 41,
    47: 42, 48: 43, 49: 44, 50: 45, 51: 46, 52: 47, 53: 48, 54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56, 62: 57, 63: 58, 64: 59, 66: 60, 69: 61,
    71: 62, 72: 63, 73: 64, 74: 65, 75: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 81: 72, 83: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 89: 79, 91: 80, 92: 81,
    93: 82, 94: 83, 95: 84, 96: 85, 97: 86, 98: 87, 99: 88, 100: 89, 101: 90, 102: 91, 103: 92, 104: 93, 105: 94, 106: 95, 107: 96, 108: 97, 109: 98, 110: 99, 111: 100,
    112: 101, 113: 102, 114: 103, 115: 104, 116: 105, 117: 106, 118: 107, 119: 108, 120: 109, 121: 110, 122: 111, 123: 112, 124: 113, 125: 114, 126: 115, 127: 116, 128: 117,
    129: 118, 130: 119, 131: 120, 132: 121, 133: 122, 134: 123, 135: 124, 136: 125, 137: 126, 138: 127, 139: 128, 140: 129, 141: 130, 142: 131, 143: 132, 144: 133, 145: 134,
    146: 135, 147: 136, 148: 137, 149: 138, 150: 139, 151: 140, 152: 141, 153: 142, 154: 143, 155: 144, 156: 145, 157: 146, 158: 147, 159: 148, 160: 149, 161: 150, 162: 151,
    163: 152, 164: 153, 165: 154, 166: 155, 167: 156, 168: 157, 169: 158, 170: 159, 171: 160, 172: 161, 173: 162, 174: 163, 175: 164, 176: 165, 177: 166, 178: 167, 179: 168,
    180: 169, 181: 170, 255: 0
}

shifted_fine_id = {}
for key, val in fine_id.items():
    val = val + 1
    shifted_fine_id[key] = val
shifted_fine_id[255] = 0
fine_id = shifted_fine_id

for key in range(182):
    if key not in fine_id:
        fine_id[key] = 0

In [4]:
fine_to_unseen = {30: 1, # frisbee
                  37: 2, # skateboard
                  89: 3, # cardboard
                  52: 4, # carrot
                  77: 5, # scissors
                  29: 6, # suitcase
                  24: 7, # giraffe
                  20: 8, # cow
                  138: 9, # road
                  161: 10, # wall-concrete
                  158: 11, # tree
                  113: 12, # grass
                  137: 13, # river
                  95: 14, # clouds
                  134: 15, # playingfield
                  }

fine_to_seen = {}

seen_idx = 1
for i in range(173):
    if i not in fine_to_unseen:
        fine_to_unseen[i] = 0
        fine_to_seen[i] = seen_idx if i != 0 else 0
        seen_idx = seen_idx + 1 if i != 0 else seen_idx
    else:
        fine_to_seen[i] = 0

In [5]:
split = 'val2017' # train2017 | val2017

In [6]:
fineseg_files = [x for x in Path(f'./COCO/labels/{split}').glob('*.png')]
fineseg_files.sort()

fineseg_files = [[line_id+1, x] for line_id, x in enumerate(fineseg_files)]

In [7]:
def return_row(line_id, fineseg_file):
    trim = False

    stem = fineseg_file.stem
    jpg_file = Path(f'./COCO/{split}/{stem}.jpg')
    
    image = Image.open(jpg_file)

    output = BytesIO()
    image.save(output, 'PNG')
    image_base64_str = base64.b64encode(output.getvalue())
    image_base64_str = image_base64_str.decode("utf-8")

    fineseg = Image.open(fineseg_file)
    fineseg_refined = np.asarray(fineseg).copy()
    mask_dict = {}
    for seg_label in np.unique(fineseg_refined.flatten()):
        mask_dict[seg_label] = fineseg_refined==seg_label

    for seg_label, mask in mask_dict.items():
        fineseg_refined[mask] = fine_id[seg_label]
        
    unseen = fineseg_refined.copy()
    mask_dict = {}
    for seg_label in np.unique(unseen.flatten()):
        mask_dict[seg_label] = unseen==seg_label
    
    for seg_label, mask in mask_dict.items():
        unseen[mask] = fine_to_unseen[seg_label]
    
    if not trim or unseen.sum() != 0:
        unseen = Image.fromarray(unseen)
        
        output = BytesIO()
        unseen.save(output, 'PNG')
        unseen_base64_str = base64.b64encode(output.getvalue())
        unseen_base64_str = unseen_base64_str.decode("utf-8")

        unseen_row = '\t'.join([image_base64_str, unseen_base64_str, stem, str(line_id)])
    else:
        unseen_row = None

    seen = fineseg_refined.copy()
    mask_dict = {}
    for seg_label in np.unique(seen.flatten()):
        mask_dict[seg_label] = seen==seg_label
    
    for seg_label, mask in mask_dict.items():
        seen[mask] = fine_to_seen[seg_label]

    if not trim or seen.sum() != 0:
        seen = Image.fromarray(seen)

        output = BytesIO()
        seen.save(output, 'PNG')
        seen_base64_str = base64.b64encode(output.getvalue())
        seen_base64_str = seen_base64_str.decode("utf-8")

        seen_row = '\t'.join([image_base64_str, seen_base64_str, stem, str(line_id)])
    else:
        seen_row = None

    return unseen_row, seen_row

class Result():
    def __init__(self):
        self.unseen_rows = []
        self.seen_rows = []

    def update_result(self, rows):
        unseen_row, seen_row = rows
        
        if unseen_row is not None:
            self.unseen_rows.append(unseen_row)

        if seen_row is not None:
            self.seen_rows.append(seen_row)

        print(f"unseen: {len(self.unseen_rows):06d} && seen: {len(self.seen_rows):06d}", end='\r')
    
    def get_rows(self):
        return self.unseen_rows, self.seen_rows

In [8]:
pool = Pool(128)
result = Result()

In [9]:
row_list = []
for line_id, fineseg_file in fineseg_files:
    pool.apply_async(return_row, args=(line_id, fineseg_file), callback=result.update_result)
pool.close()
pool.join()

unseen_rows, _ = result.get_rows()

unseen_rows.sort(key=lambda x: int(x.split('\t')[-1]))
unseen_fullfile = "\n".join(unseen_rows)

with open(f'./dataset/coco/unseen_{split}.tsv', 'w') as f:
    f.write(unseen_fullfile)